# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [42]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import numpy as np
# Import API key
from api_keys import geoapify_key

In [28]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../Weatherpy/output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,xiongzhou,25.1167,114.3000,31.17,68,1,1.28,CN,1724638617
1,1,sayula de aleman,17.8667,-94.9500,24.73,89,95,1.40,MX,1724638618
2,2,port-aux-francais,-49.3500,70.2167,4.16,85,90,15.55,TF,1724638619
3,3,whitehorse,60.7161,-135.0538,14.43,67,75,7.72,CA,1724638487
4,4,sydney,-33.8679,151.2073,24.69,32,0,0.89,AU,1724638492


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [29]:
map_plot = city_data_df.hvplot.points(
    x='Lng', 
    y='Lat', 
    geo=True, 
    size='Humidity',        # Size of the points based on Humidity
    color='Humidity',       # Color points based on Humidity
    colormap='rainbow',      # Use the 'rainbow' colormap for better contrast
    tiles='OSM',            # Use OpenStreetMap tiles as background
    alpha=0.7,              # Set transparency
    width=800,              # Set plot width
    height=600,             # Set plot height
    hover_cols=['City'],    # Show city names on hover
    title="City Locations and Humidity Levels"
)

# Display the plot
map_plot


:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (Humidity,City)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [54]:
# Narrow down cities that fit criteria and drop any results with null values
my_requirements = city_data_df[(city_data_df['Max Temp']>= 27) & (city_data_df['Max Temp']< 29) & (city_data_df['Humidity'] < 75) &
(city_data_df['Humidity'] > 70) & (city_data_df['Cloudiness'] > 50)]

# Drop any rows with null values
my_requirements.dropna(inplace = True) 

# Display sample data
my_requirements

/var/folders/c4/yw02kkx51lj10t5f6cbj7_pr0000gn/T/ipykernel_18703/3045628319.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  my_requirements.dropna(inplace = True)


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
58,58,wailua homesteads,22.0669,-159.3780,27.44,72,100,8.75,US,1724638685
89,89,sinjai,-5.1241,120.2530,28.34,71,63,3.70,ID,1724638720
96,96,angaur state,6.9060,134.1300,28.62,73,65,2.09,PW,1724638728
174,174,hithadhoo,-0.6000,73.0833,28.34,72,100,3.51,MV,1724638824
194,194,tofol,5.3248,163.0078,28.50,71,96,1.26,FM,1724638846
321,321,dapa,9.7594,126.0531,28.48,74,100,4.22,PH,1724638998
340,340,kudahuvadhoo,2.6708,72.8944,27.90,73,96,1.80,MV,1724639023
543,543,cooper's town,26.8667,-77.5167,28.91,74,100,5.54,BS,1724639267


### Step 3: Create a new DataFrame called `hotel_df`.

In [55]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df= my_requirements[['City' , 'Country', 'Lat', 'Lng', 'Humidity']].copy()
hotel_df
# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = np.nan
hotel_df
# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
58,wailua homesteads,US,22.0669,-159.3780,72,NaN
89,sinjai,ID,-5.1241,120.2530,71,NaN
96,angaur state,PW,6.9060,134.1300,73,NaN
174,hithadhoo,MV,-0.6000,73.0833,72,NaN
194,tofol,FM,5.3248,163.0078,71,NaN
321,dapa,PH,9.7594,126.0531,74,NaN
340,kudahuvadhoo,MV,2.6708,72.8944,73,NaN
543,cooper's town,BS,26.8667,-77.5167,74,NaN


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [56]:
# Set the search radius for finding hotels (in meters)
radius = 10000  # 10 kilometers

# Set up the parameters dictionary 
params = {
    "categories": "accommodation.hotel",  # Search for hotels
    "limit": 1,  # We only want the nearest hotel
    "apiKey": geoapify_key  
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # Get latitude, longitude from the DataFrame
    lat = row['Lat']
    lng = row['Lng']

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lng},{lat},{radius}"
    params["bias"] = f"proximity:{lng},{lat}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make an API request using the params dictionary
    response = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = response.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found"
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df


Starting hotel search


/var/folders/c4/yw02kkx51lj10t5f6cbj7_pr0000gn/T/ipykernel_18703/1644269305.py:35: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'Hilton Garden Inn Kauai Wailua Bay' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]


wailua homesteads - nearest hotel: Hilton Garden Inn Kauai Wailua Bay
sinjai - nearest hotel: Hotel Sinjai
angaur state - nearest hotel: No hotel found
hithadhoo - nearest hotel: Pebbles Inn
tofol - nearest hotel: Nautilus Resort
dapa - nearest hotel: Island Suites
kudahuvadhoo - nearest hotel: Hotel Riu Palace Maldivas
cooper's town - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
58,wailua homesteads,US,22.0669,-159.3780,72,Hilton Garden Inn Kauai Wailua Bay
89,sinjai,ID,-5.1241,120.2530,71,Hotel Sinjai
96,angaur state,PW,6.9060,134.1300,73,No hotel found
174,hithadhoo,MV,-0.6000,73.0833,72,Pebbles Inn
194,tofol,FM,5.3248,163.0078,71,Nautilus Resort
321,dapa,PH,9.7594,126.0531,74,Island Suites
340,kudahuvadhoo,MV,2.6708,72.8944,73,Hotel Riu Palace Maldivas
543,cooper's town,BS,26.8667,-77.5167,74,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [58]:
%%capture --no-display

# Configure the map plot
map_plot = hotel_df.hvplot.points(
    x='Lng', 
    y='Lat', 
    geo=True, 
    size='Humidity',        # Size of the points based on Humidity
    color='Humidity',       # Color points based on Humidity
    colormap='rainbow',      # Use the 'rainbow' colormap for better contrast
    tiles='OSM',            # Use OpenStreetMap tiles as background
    alpha=0.7,              # Set transparency
    width=800,              # Set plot width
    height=600,             # Set plot height
    hover_cols=['City', 'Country', 'Hotel Name'],  # Additional hover information
    title="City Locations, Humidity Levels, and Nearby Hotels"
)

# Display the plot
map_plot


:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (Humidity,City,Country,Hotel Name)